In [23]:
import numpy as np
import time
import cv2
import os
import imutils

In [24]:
image_path = '../Assets/images/predict-2.jpeg'
video_path = '../Assets/videos/vehicles1.mp4'

In [25]:
def detectFeatures(frame, net, ln, Labels,bicycleIndx=0, carIndx=0, motorbikeIndx=0, busIndx=0, truckIndx=0):
    
    (H, W) = frame.shape[:2]

    #We'll store out predicitons with it's details such as confidence, bounding box points in results
    results=[]
    
    # construct a blob from the input frame and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes
    # and associated probabilities
    blob=cv2.dnn.blobFromImage(frame, 1/255.0,(416,416), swapRB=True, crop=False)
    
    
    #Giving the blob as input to our model to process this frame
    net.setInput(blob)
    start = time.time()
    #Getting the output of our Model which contains predictions
    layerOutputs=net.forward(ln)
    end = time.time()
    
    #We'll Stores the bounding box co-ordinates, confidences, ID which corresponds to object it is in these for our predictions
    boxes=[]
    confidences=[]
    classIDs=[]
    classname = []
    #Iterate though our outputs and each detection in each output
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            #Extract Confidence for each detection and it's ID for item it corresponds to
            scores=detection[5:]
            classID=np.argmax(scores)
            confidence=scores[classID]
            
            #We just want Bicycles, Cars, Bikes, Trucks and Buses to be detected
            if ((classID==bicycleIndx or classID==carIndx or classID==motorbikeIndx or classID==busIndx or classID==truckIndx) and confidence>0.5):
                #Get the boundary box co-ordinates for our Detectection
                box=detection[0:4]*np.array([W,H,W,H])
                (centerX, centerY, width, height)=box.astype('int')
                x=int(centerX-(width/2))
                y=int(centerY-(height/2))
                
                #Save these co-ordinates, confidence, ID of item number in the lists we initialized
                boxes.append([x,y,int(width),int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                classname.append(Labels[classID])
                
    #Applying Non-Max Suppression to remove extra Bounding Boxes
    indxs=cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.5)
    
#     boxes = [boxes[i] for i in indxs]
#     classname = [classname[i] for i in indxs]
#     confidences = [confidences[i] for i in indxs]
#     classIDs = [classIDs[i] for i in indxs]
    
    
    #Storing Final Detection's features in results
    if len(indxs)>0:
        for i in indxs.flatten():
            (x,y)=(boxes[i][0], boxes[i][1])
            (w,h)=(boxes[i][2],boxes[i][3])
            r=(confidences[i], (x,y,x+w,y+h),classIDs[i],classname[i])
            results.append(r)
#     print(results)
    cv2.imshow('Frame',frame)
    cv2.waitKey(0)
    return results

In [26]:
def get_vehicle_count(boxes, class_names,list_of_vehicles):
    
    total_vehicle_count = 0 # total vechiles present in the image
    dict_vehicle_count = {} # dictionary with count of each distinct vehicles detected
    
    for i in range(len(boxes)):

            class_name = class_names[i]
       
            if(class_name in list_of_vehicles):
                total_vehicle_count += 1
                dict_vehicle_count[class_name] = dict_vehicle_count.get(class_name,0) + 1

    return total_vehicle_count, dict_vehicle_count

In [27]:
def customYolo(frame):
    
    
    # setting the paths
    coco_names_path = '../Assets/yolo-coco/custom/classes.names'
    weightsPath = '../Assets/yolo-coco/custom/Training-2/yolov3_custom_last_6000.weights'
    configPath = '../Assets/yolo-coco/custom/yolov3_custom.cfg'
    
    
    # load the COCO class labels our YOLO model was trained on
    Labels = []
    with open(coco_names_path,'r',encoding='utf8') as f:
        Labels = [line.strip() for line in f.readlines()]
#     print(Labels)
    
    
    # initialize a list of colors to represent each possible class label
    np.random.seed(42)
    COLORS = np.random.randint(0, 255, size=(len(Labels), 3),dtype="uint8")
    
    
    # load our YOLO object detector trained on COCO dataset (80 classes)
    # and determine only the *output* layer names that we need from YOLO
    net = cv2.dnn.readNetFromDarknet(configPath,weightsPath)
    layers_names = net.getLayerNames()
    output_layers = [layers_names[i - 1] for i in net.getUnconnectedOutLayers()]
    
    
#     list_of_vehicles = ['car', 'truck', 'bus', 'motorbike', 'bicycle']
    list_of_vehicles = ['car', 'truck', 'bus', 'motorbike', 'bicycle', 'ambulance', 'fire engine','auto rickshaw']
   
    # if the frame dimensions are empty, grab them
#     if W is None or H is None:
#     print(frame.shape)
    (H, W) = frame.shape[:2]
    
#     print(frame.shape)
#     cv2.imshow("frame",frame)
#     cv2.waitKey(0)
    # construct a blob from the input frame and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes
    # and associated probabilities
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(output_layers)
    end = time.time()

    # initialize our lists of detected bounding boxes, confidences,
    # and class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []
    classname = []
#     print(layerOutputs)
    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability)
            # of the current object detection
          
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > 0.5:
                # scale the bounding box coordinates back relative to
                # the size of the image, keeping in mind that YOLO
                # actually returns the center (x, y)-coordinates of
                # the bounding box followed by the boxes' width and
                # height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates,
                # confidences, and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                classname.append(Labels[classID])

    # apply non-maxima suppression to suppress weak, overlapping
    # bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.5)
    results = []
    if len(idxs)>0:
        for i in idxs.flatten():
            (x,y)=(boxes[i][0], boxes[i][1])
            (w,h)=(boxes[i][2],boxes[i][3])
            r=(confidences[i], (x,y,x+w,y+h),classIDs[i],classname[i])
            results.append(r)

    boxes = [boxes[i] for i in idxs]
    classname = [classname[i] for i in idxs]

    total_vehicles, each_vehicle = get_vehicle_count(boxes, classname,list_of_vehicles)


    return each_vehicle,results

    
    

In [28]:
def preTrainedYolo():
    
    # setting the paths
    coco_names_path = '../Assets/yolo-coco/coco.names'
    weightsPath = '../Assets/yolo-coco/yolov3.weights'
    configPath = '../Assets/yolo-coco/yolov3.cfg'    
    
    
    # load the COCO class labels our YOLO model was trained on
    Labels = []
    with open(coco_names_path,'r',encoding='utf8') as f:
        Labels = [line.strip() for line in f.readlines()]
#     print(Labels)
    
    
    list_of_vehicles = ['car', 'truck', 'bus', 'motorbike', 'bicycle', 'ambulance', 'fire engine','auto rickshaw']
   
    # initialize a list of colors to represent each possible class label
    np.random.seed(42)
    COLORS = np.random.randint(0, 255, size=(len(list_of_vehicles), 3),dtype="uint8")
    
    
    # load our YOLO object detector trained on COCO dataset (80 classes)
    # and determine only the *output* layer names that we need from YOLO
    net = cv2.dnn.readNetFromDarknet(configPath,weightsPath)
    layers_names = net.getLayerNames()
    output_layers = [layers_names[i - 1] for i in net.getUnconnectedOutLayers()]
    
    # initialize the video stream, pointer to output video file, and
    # frame dimensions
    vs = cv2.VideoCapture(image_path)
    writer = None
    (W, H) = (None, None)
    fps=vs.get(cv2.CAP_PROP_FPS)
    
#     # try to determine the total number of frames in the video file
#     try:
#         prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
#             else cv2.CAP_PROP_FRAME_COUNT
#         total = int(vs.get(prop))
#         print("[INFO] {} total frames in video".format(total))

#     # an error occurred while trying to determine the total
#     # number of frames in the video file
#     except:
#         print("[INFO] could not determine # of frames in video")
#         print("[INFO] no approx. completion time can be provided")
#         total = -1
    
#     tempPath='../Assets/images/EmergencyVehicleClassificationFastAI/1.jpg'
    
    while True:
        #Reading frame from video
        (grabbed, frame) = vs.read()

        # if the frame was not grabbed, then we have reached the end
        # of the stream
        if not grabbed:
            break
        
        
#         frame = imutils.resize(frame, width=400)
        
        #Extracting individual bounding boxes
        results = detectFeatures(frame, net, output_layers,Labels,bicycleIndx=Labels.index('bicycle'), carIndx=Labels.index('car'), motorbikeIndx=Labels.index('motorbike'), busIndx=Labels.index('bus'), truckIndx=Labels.index('truck'))
        #Setting Number of Vehicles initially 0
        vehicles=0
        emergency = 0
        urgent=False
        active=False
        dict_vehicles = {}
        
        #Iterating through our results in Current frame
        if(results):
            # results list is not empty
            for (i,(prob, bbox, classID, name)) in enumerate(results):

                #Getting co-ordinates our current detection, and adding to corresponding Detection we got
                real_name = name
                (startX, startY, endX, endY) = bbox
                if name=='car' or name=='bus' or name=='truck':

                    vehicle=frame[startY:endY, startX:endX]
                    
                    (H,W) = vehicle.shape[:2]
                    if H==0 or W==0:
                        continue
                        
                    each_vehicle,_ = customYolo(vehicle)


                    if(each_vehicle):
                        for key,value in each_vehicle.items():

                            dict_vehicles[key] = dict_vehicles.get(key,0) + value

                            if((key=='ambulance' or key=='fire engine') and active==True):
                                urgent = True

                            real_name = key
                    else:
                        dict_vehicles[name] = dict_vehicles.get(name,0) + 1

                elif name=='bicycle' or name=='motorbike':
                    dict_vehicles[name] = dict_vehicles.get(name,0) + 1


                #Plotting rectangle around the detection and displaying what it is                                                     
                color = [int(c) for c in COLORS[list_of_vehicles.index(real_name)]]
                cv2.rectangle(frame, (startX,startY),(endX,endY),color,2)
                cv2.putText(frame, real_name, (startX,startY-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)
        
        else:
            
            _,res = customYolo(frame)
            
            (H,W) = frame.shape[:2]
            
            if H==0 or W==0:
                pass
            else:
                for (i,(prob, bbox, classID, name)) in enumerate(res):

                    (startX, startY, endX, endY) = bbox
                    dict_vehicles[name] = dict_vehicles.get(name,0) + 1  
                    #Plotting rectangle around the detection and displaying what it is                                                     
                    color = [int(c) for c in COLORS[list_of_vehicles.index(name)]]
                    cv2.rectangle(frame, (startX,startY),(endX,endY),color,2)
                    cv2.putText(frame, name, (startX,startY-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)
        
        
        print(dict_vehicles)
        cv2.imshow('Frame',frame)
        cv2.waitKey(0)
        
        if writer is None:
            fourcc=cv2.VideoWriter_fourcc(*'MJPG')
            writer=cv2.VideoWriter('../Assets/images/predictedResult.jpeg', fourcc, 30, (frame.shape[1],frame.shape[0]), True)

        writer.write(frame)

        
    writer.release()
    vs.release()

In [29]:
preTrainedYolo()

{'car': 11, 'auto rickshaw': 11, 'bus': 1, 'ambulance': 1, 'bicycle': 13, 'motorbike': 9}
